In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
)

from langchain.agents import initialize_agent, load_tools, AgentType
tools = load_tools(["wikipedia"])
agent = initialize_agent(
	tools=tools,
	llm=llm,
	agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)
agent.invoke("결핵 중 신장결핵을 진단하기 위해 꼭 필요한 검사를 알려줘. 필요하면 영어로 검색해")



> Entering new AgentExecutor chain...
신장결핵의 진단을 위해 필요한 검사에 대한 정보를 찾기 위해 영어로 검색할 필요가 있다. 신장결핵에 의한 감염을 발견하는 데 일반적으로 수행되는 검사 방법을 확인하겠다.
Action: wikipedia
Action Input: renal tuberculosis diagnosis
Observation: Page: Diagnosis of tuberculosis
Summary: Tuberculosis is diagnosed by finding Mycobacterium tuberculosis bacteria in a clinical specimen taken from the patient.  While other investigations may strongly suggest tuberculosis as the diagnosis, they cannot confirm it.
A complete medical evaluation for tuberculosis (TB) must include a medical history, a physical examination, a chest X-ray and microbiological examination (of sputum or some other appropriate sample).  It may also include a tuberculin skin test, other scans and X-rays, surgical biopsy.

Page: Extrapulmonary tuberculosis
Summary: Extrapulmonary tuberculosis is tuberculosis (TB) within a location in the body other than the lungs. It accounts for an increasing fraction of active cases, from 20 to 40% according to published r

{'input': '결핵 중 신장결핵을 진단하기 위해 꼭 필요한 검사를 알려줘. 필요하면 영어로 검색해',
 'output': '신장결핵을 진단하기 위해서는 미생물학적 검사와 함께 영상 검사(예: 초음파나 CT 스캔)가 꼭 필요하다.'}

In [ ]:
from langchain.tools import tool

@tool
def calculate_fena(input_str: str) -> float:
    """
    FeNa (%) = (Urine_Na × Plasma_Cr) / (Plasma_Na × Urine_Cr) × 100
    
    input_str는 다음과 같은 형식이어야 합니다:
    "urine_na=20, plasma_na=140, urine_cr=50, plasma_cr=2"
    """
    try:
        parts = dict(
            item.strip().split("=")
            for item in input_str.split(",")
        )
        urine_na = float(parts["urine_na"])
        plasma_na = float(parts["plasma_na"])
        urine_cr = float(parts["urine_cr"])
        plasma_cr = float(parts["plasma_cr"])
        
        fena = (urine_na * plasma_cr) / (plasma_na * urine_cr) * 100
        return round(fena, 2)
    except Exception as e:
        return -1.0  # 실패 시 -1 반환
from langchain.memory import ConversationBufferMemory

tools = [calculate_fena]
memory = ConversationBufferMemory(memory_key="chat_history")
converstaional_agent = initialize_agent(
    agent = 'conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    memory=memory
)
converstaional_agent.invoke("신장결핵을 진단하기 위해 필요한 검사 중 하나는 FENa를 계산하는 것입니다. urine_na=20, plasma_na=140, urine_cr=50, plasma_cr=2일 때 FENa를 계산해줘")



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: calculate_fena
Action Input: "urine_na=20, plasma_na=140, urine_cr=50, plasma_cr=2"
```
Observation: -1.0
Thought:Do I need to use a tool? No  
AI: FENa (Fractional Excretion of Sodium) is calculated using the formula: 

\[
FENa (\%) = \left(\frac{{\text{{Urine Na}} \times \text{{Plasma Cr}}}}{{\text{{Plasma Na}} \times \text{{Urine Cr}}}}\right) \times 100
\]

Plugging in the values you provided:

\[
FENa = \left(\frac{{20 \times 2}}{{140 \times 50}}\right) \times 100 = \left(\frac{{40}}{{7000}}\right) \times 100 \approx 0.57\%
\]

So, the calculated FENa is approximately 0.57%.

> Finished chain.


{'input': '신장결핵을 진단하기 위해 필요한 검사 중 하나는 FENa를 계산하는 것입니다. 예를 들어, urine_na=20, plasma_na=140, urine_cr=50, plasma_cr=2일 때 FENa를 계산해줘',
 'chat_history': '',
 'output': 'FENa (Fractional Excretion of Sodium) is calculated using the formula: \n\n\\[\nFENa (\\%) = \\left(\\frac{{\\text{{Urine Na}} \\times \\text{{Plasma Cr}}}}{{\\text{{Plasma Na}} \\times \\text{{Urine Cr}}}}\\right) \\times 100\n\\]\n\nPlugging in the values you provided:\n\n\\[\nFENa = \\left(\\frac{{20 \\times 2}}{{140 \\times 50}}\\right) \\times 100 = \\left(\\frac{{40}}{{7000}}\\right) \\times 100 \\approx 0.57\\%\n\\]\n\nSo, the calculated FENa is approximately 0.57%.'}

In [16]:
from pydantic import BaseModel
from langchain.tools import StructuredTool

# 1. interpret_creatinine_tool
class CrInput(BaseModel):
    gender: str
    age: int
    cr: float

def interpret_creatinine_func(data: CrInput) -> str:
    gender, age, cr = data.gender, data.age, data.cr
    adjusted_cr = cr + 0.5 if gender.lower() == "female" else cr
    if adjusted_cr <= 1.0:
        eGFR = 90
    elif adjusted_cr <= 1.5:
        eGFR = 60
    elif adjusted_cr <= 2.0:
        eGFR = 40
    elif adjusted_cr <= 3.0:
        eGFR = 30
    elif adjusted_cr <= 4.0:
        eGFR = 15
    else:
        eGFR = 10
    if eGFR >= 60:
        interpretation = "정상 또는 경증 신기능 저하"
    elif eGFR >= 30:
        interpretation = "중등도 저하"
    elif eGFR >= 15:
        interpretation = "중증 저하"
    else:
        interpretation = "ESRD 가능성"
    return f"eGFR: 약 {eGFR}, 해석: {interpretation}"

interpret_creatinine_tool = StructuredTool.from_function(
    func=interpret_creatinine_func,
    name="interpret_creatinine",
    description="성별, 나이, Cr 수치로 eGFR 추정 및 해석"
)

# 2. calculate_bun_cr_ratio_tool
class BunCrInput(BaseModel):
    bun: float
    cr: float

def calculate_bun_cr_ratio_func(data: BunCrInput) -> str:
    ratio = data.bun / data.cr
    if ratio > 20:
        reason = "탈수, GI 출혈, 단백질 과다섭취 등"
        if data.cr<=0.5:
            reason = "영양실조 등"
    elif ratio > 10:
        reason = "정상 또는 경증 증가"
    else:
        reason = "간기능 저하, SIADH 또는 근육 손상 가능"
    return f"BUN/Cr 비율: {ratio:.1f}, 해석: {reason}"

calculate_bun_cr_ratio_tool = StructuredTool.from_function(
    func=calculate_bun_cr_ratio_func,
    name="calculate_bun_cr_ratio",
    description="BUN/Cr 비율을 해석하여 임상적 원인을 추정"
)

# 3. interpret_urinalysis_tool
class UrinalysisInput(BaseModel):
    SG: float
    pH: float
    glucose: str
    ketone: str
    nitrite: str
    leukocyte_esterase: str
    acidosis: str

def interpret_urinalysis_func(data: UrinalysisInput) -> str:
    results = []
    if data.SG <= 1.003:
        results.append("희석된 소변")
    elif data.SG >= 1.030:
        results.append("농축된 소변 또는 glucose/조영제 존재 가능")
    if data.pH < 5:
        results.append("H+ 배출 정상")
    elif data.pH > 5 and data.acidosis == "yes":
        results.append("산증 동반 + pH > 5 → RTA 의심")
    if data.glucose == "pos":
        results.append("혈당 > 180mg/dL 또는 근위세뇨관 문제")
    if data.ketone == "pos":
        results.append("케톤산증 또는 금식")
    if data.nitrite == "pos":
        results.append("Enterobacteriaceae에 의한 UTI 의심")
    if data.leukocyte_esterase == "pos":
        results.append("백혈구 존재 → UTI 또는 염증")
    return "\n".join(results) if results else "특이 소견 없음"

interpret_urinalysis_tool = StructuredTool.from_function(
    func=interpret_urinalysis_func,
    name="interpret_urinalysis",
    description="소변검사 결과를 해석하여 이상소견 및 임상적 의미 파악"
)

# 4. rta_assessment_tool
class RTAInput(BaseModel):
    ph: float
    has_acidosis: bool

def rta_assessment_func(data: RTAInput) -> str:
    if not data.has_acidosis:
        return "산증이 없어 RTA 평가 대상 아님"
    if data.ph > 5.5:
        return "distal RTA (Type 1) 가능성"
    else:
        return "산 배출은 정상"

rta_assessment_tool = StructuredTool.from_function(
    func=rta_assessment_func,
    name="rta_assessment_tool",
    description="소변 pH와 산증 여부로 RTA 가능성 평가"
)

# 5. pyuria_analysis_tool
class PyuriaInput(BaseModel):
    wbc_count: int
    nitrite: str
    leukocyte_esterase: str
    bacteria: str

def pyuria_analysis_func(data: PyuriaInput) -> str:
    reasons = []
    if data.wbc_count > 10:
        reasons.append("농뇨 있음")
    if data.nitrite == "pos":
        reasons.append("Nitrite 양성 → UTI 의심")
    if data.leukocyte_esterase == "pos":
        reasons.append("Leukocyte esterase 양성 → 염증")
    if data.bacteria == "pos":
        reasons.append("세균 검출 → 감염 가능성 높음")
    return "\n".join(reasons) if reasons else "농뇨 또는 감염 관련 이상 없음"

pyuria_analysis_tool = StructuredTool.from_function(
    func=pyuria_analysis_func,
    name="pyuria_analysis",
    description="농뇨 관련 수치로 UTI 가능성 해석"
)

# 6. epithelial_cell_analysis_tool
class EpithelialInput(BaseModel):
    cell_type: str

def epithelial_cell_analysis_func(data: EpithelialInput) -> str:
    if data.cell_type == "squamous":
        return "오염된 소변 가능성"
    elif data.cell_type == "renal_tubular":
        return "ATN 또는 세관사이질질환 가능성"
    elif data.cell_type == "granular_cast":
        return "급성 세뇨관 괴사 가능성"
    else:
        return "특정 소견 없음"

epithelial_cell_analysis_tool = StructuredTool.from_function(
    func=epithelial_cell_analysis_func,
    name="epithelial_cell_analysis",
    description="소변 내 세포 종류에 따른 임상적 의미 해석"
)

# 7. vitamin_c_interference_tool
class VitaminCInput(BaseModel):
    vit_c: bool

def vitamin_c_interference_func(data: VitaminCInput) -> str:
    if data.vit_c:
        return "비타민 C → dipstick 검사 위음성 가능 (heme, glucose 등)"
    else:
        return "비타민 C 복용 없음 → 위음성 가능성 낮음"

vitamin_c_interference_tool = StructuredTool.from_function(
    func=vitamin_c_interference_func,
    name="vitamin_c_interference",
    description="비타민 C가 소변검사에 미치는 영향 분석"
)

# 8. educational_lookup_tool
class LookupInput(BaseModel):
    query: str

def educational_lookup_func(data: LookupInput) -> str:
    query = data.query.lower()
    if "출혈" in query and "bun" in query:
        return "GI 출혈 → 단백분해로 BUN 증가"
    elif "cystatin" in query:
        return "Cystatin C는 Cr보다 더 정확한 GFR 지표"
    else:
        return "해당 주제에 대한 교육 정보 없음"

educational_lookup_tool = StructuredTool.from_function(
    func=educational_lookup_func,
    name="educational_lookup",
    description="신장 관련 의학 개념 설명을 제공"
)

# 툴 리스트
renal_tools = [
    interpret_creatinine_tool,
    calculate_bun_cr_ratio_tool,
    interpret_urinalysis_tool,
    rta_assessment_tool,
    pyuria_analysis_tool,
    epithelial_cell_analysis_tool,
    vitamin_c_interference_tool,
    educational_lookup_tool
]




In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
	[
		("system", "You are a helpful assistant"),
		MessagesPlaceholder("chat_history", optional=True),
		("human", "{input}"),
		MessagesPlaceholder("agent_scratchpad"),
	]
)
from langchain.agents import AgentExecutor, create_openai_functions_agent

# 1. LLM 준비 (gpt-4 or gpt-3.5-turbo)
llm = ChatOpenAI(model="gpt-4", temperature=0)

# 2. Agent 생성 (LLM + Tools)
agent = create_openai_functions_agent(
    llm=llm,
    tools=renal_tools,
    prompt=prompt,  # 기본 프롬프트 사용
)

# 3. AgentExecutor 준비
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=renal_tools,
    verbose=True
)
questions = [
    "70세 여성이고 Cr이 1.4인데 신기능 괜찮나요?",
    "BUN이 40, Cr이 1.2인 환자가 있습니다. 이 환자의 원인은 무엇일까?",
    "산증인데 소변 pH가 6이야. 문제 있을까?",
    "왜 위장관 출혈이 BUN을 증가시킬까?",
 
]

# 4. 실행
for question in questions:
    print('\n\n')
    result = agent_executor.invoke({"input": question})
    print()
    print(result['output'])







> Entering new AgentExecutor chain...

Invoking: `interpret_creatinine` with `{'data': {'gender': 'female', 'age': 70, 'cr': 1.4}}`


eGFR: 약 40, 해석: 중등도 저하해당 환자의 eGFR(추정 사구체 여과율)은 약 40으로, 이는 신기능이 중등도로 저하되었음을 의미합니다. 이는 신장이 체내에서 노폐물을 제거하는 능력이 일정 부분 감소하였음을 나타냅니다. 이에 따라 추가적인 검사나 치료가 필요할 수 있으니, 전문적인 의료 상담을 받아보시는 것이 좋습니다.

> Finished chain.

해당 환자의 eGFR(추정 사구체 여과율)은 약 40으로, 이는 신기능이 중등도로 저하되었음을 의미합니다. 이는 신장이 체내에서 노폐물을 제거하는 능력이 일정 부분 감소하였음을 나타냅니다. 이에 따라 추가적인 검사나 치료가 필요할 수 있으니, 전문적인 의료 상담을 받아보시는 것이 좋습니다.





> Entering new AgentExecutor chain...

Invoking: `calculate_bun_cr_ratio` with `{'data': {'bun': 40, 'cr': 1.2}}`


BUN/Cr 비율: 33.3, 해석: 탈수, GI 출혈, 단백질 과다섭취 등이 환자의 BUN/Cr 비율은 33.3으로, 이는 탈수, 위장관 출혈, 단백질 과다섭취 등이 원인일 수 있음을 나타냅니다. 이러한 상황에서는 환자의 증상, 식사 패턴, 최근의 약물 사용 등을 포함한 전반적인 임상 상황을 고려해야 합니다.

> Finished chain.

이 환자의 BUN/Cr 비율은 33.3으로, 이는 탈수, 위장관 출혈, 단백질 과다섭취 등이 원인일 수 있음을 나타냅니다. 이러한 상황에서는 환자의 증상, 식사 패턴, 최근의 약물 사용 등을 포함한 전반적인 임상 상황을 고려해야 합니다.





> Entering new AgentExecutor chain...



In [23]:
from pydantic import BaseModel
from langchain.tools import StructuredTool

# -----------------------------
# 1. Dipstick 기반 초기 스크리닝
# -----------------------------
class DipstickInput(BaseModel):
    dipstick: str  # "-", "trace", "1+", ..., "4+"

def screen_proteinuria_dipstick_func(data: DipstickInput) -> str:
    if data.dipstick in ["2+", "3+", "4+"]:
        return f"{data.dipstick}는 의미 있는 단백뇨일 가능성이 있어 정량검사가 필요합니다."
    elif data.dipstick in ["1+", "trace"]:
        return f"{data.dipstick}는 경도의 단백뇨로, 임상 상황에 따라 추적이 필요할 수 있습니다."
    elif data.dipstick == "-":
        return "단백뇨는 확인되지 않았습니다."
    else:
        return "알 수 없는 dipstick 결과입니다."

screen_proteinuria_dipstick_tool = StructuredTool.from_function(
    func=screen_proteinuria_dipstick_func,
    name="screen_proteinuria_dipstick",
    description="소변 단백질 정성검사(dipstick)를 기반으로 단백뇨 여부 및 정량검사 필요성 판단"
)

# -----------------------------
# 2. UPCR / UACR로 단백뇨 분류
# -----------------------------
class ProteinuriaInput(BaseModel):
    upcr: float = None  # mg/gCr
    uacr: float = None  # mg/gCr

def classify_proteinuria_func(data: ProteinuriaInput) -> str:
    if data.uacr is not None:
        if data.uacr < 30:
            return "정상 범위의 albuminuria입니다."
        elif 30 <= data.uacr <= 300:
            return "Moderately increased albuminuria (microalbuminuria)입니다."
        else:
            return "Severely increased albuminuria (macroalbuminuria)입니다."
    elif data.upcr is not None:
        if data.upcr < 150:
            return "정상 범위의 단백뇨입니다."
        elif 150 <= data.upcr <= 3500:
            return "의미 있는 단백뇨가 있으며, nephrotic 수준은 아닙니다."
        else:
            return "Nephrotic-range proteinuria입니다."
    else:
        return "UPCR 또는 UACR 값이 필요합니다."

classify_proteinuria_tool = StructuredTool.from_function(
    func=classify_proteinuria_func,
    name="classify_proteinuria",
    description="UPCR 또는 UACR을 기반으로 단백뇨 수준을 분류합니다. 단위는 mg/gCr입니다."
)

# -----------------------------
# 3. 일시적/기립성 단백뇨 감별
# -----------------------------
class TransientOrthostaticInput(BaseModel):
    transient_trigger: bool  # fever, exercise 등
    posture_test_result: str = None  # "orthostatic", "non-orthostatic", "unknown"

def check_transient_or_orthostatic_func(data: TransientOrthostaticInput) -> str:
    if data.transient_trigger:
        return "일시적 단백뇨일 가능성이 있으며, 원인 회복 시 자연 소실됩니다."
    if data.posture_test_result == "orthostatic":
        return "기립성 단백뇨로 확인되어 경과관찰 가능합니다."
    elif data.posture_test_result == "non-orthostatic":
        return "기립성 단백뇨는 아닙니다."
    return "일시성 또는 기립성과 관련된 정보가 부족합니다."

check_transient_or_orthostatic_tool = StructuredTool.from_function(
    func=check_transient_or_orthostatic_func,
    name="check_transient_or_orthostatic_proteinuria",
    description="일시적 단백뇨 또는 기립성 단백뇨 여부 감별"
)

# -----------------------------
# 4. 원인 감별 (사구체성 등)
# -----------------------------
from typing import Optional, Literal
from pydantic import BaseModel

class EtiologyInput(BaseModel):
    hematuria_status: Literal["있음", "없음", "불명확"]
    upep_pattern: Literal["albumin dominant", "light chain dominant", "mixed", "unknown", "검사 안함"]
    notes: Optional[str] = None  # 자유서술

def suggest_proteinuria_etiology_func(data: EtiologyInput) -> str:
    # UPEP 기반 원인 감별
    if data.upep_pattern == "albumin dominant":
        origin = "사구체성 단백뇨"
        comment = "사구체 여과장벽 손상을 시사합니다."
    elif data.upep_pattern == "light chain dominant":
        origin = "과부하성 단백뇨"
        comment = "예: 다발성 골수종 등에서 나타납니다."
    elif data.upep_pattern == "mixed":
        origin = "혼합형 단백뇨"
        comment = "사구체와 세뇨관 손상이 모두 있을 수 있습니다."
    elif data.upep_pattern == "unknown":
        origin = "원인 미상 단백뇨"
        comment = "UPEP 결과 해석이 어려워 추가 검사가 필요합니다."
    else:  # "검사 안함"
        origin = "단백뇨 원인 평가 불가"
        comment = "UPEP 검사 없이 정확한 병인 감별은 어렵습니다."

    # 혈뇨 여부 해석
    if data.hematuria_status == "있음":
        hematuria_comment = "혈뇨가 동반되어 사구체 손상의 가능성이 높습니다."
    elif data.hematuria_status == "없음":
        hematuria_comment = "혈뇨는 없어 사구체성 가능성은 상대적으로 낮을 수 있습니다."
    else:
        hematuria_comment = "혈뇨 여부가 불명확하여 해석에 한계가 있습니다."

    notes_text = f"\n[참고 사항] {data.notes}" if data.notes else ""

    return f"{origin} 가능성이 있습니다. {comment} {hematuria_comment}{notes_text}"

# StructuredTool 등록
suggest_proteinuria_etiology_tool = StructuredTool.from_function(
    func=suggest_proteinuria_etiology_func,
    name="suggest_proteinuria_etiology",
    description="UPEP 결과 및 혈뇨 여부로 단백뇨의 병인 감별"
)


# -----------------------------
# 5. 종합 설명 생성
# -----------------------------
class ProteinuriaSummaryInput(BaseModel):
    age: int
    gender: str  # "male" / "female"
    dipstick: str
    upcr: float = None
    hematuria: bool = False

def explain_proteinuria_summary_func(data: ProteinuriaSummaryInput) -> str:
    level = "경도" if data.dipstick in ["1+", "trace"] else "중등도 이상"
    nephrotic = data.upcr and data.upcr > 3500
    sex = "남성" if data.gender == "male" else "여성"

    summary = f"{data.age}세 {sex} 환자에서 dipstick {data.dipstick} 검사 결과 {level} 단백뇨로 보입니다."
    if data.upcr is not None:
        summary += f" UPCR {data.upcr} mg/gCr로, "
        summary += "nephrotic 수준의 단백뇨입니다." if nephrotic else "비신증후군 수준의 단백뇨입니다."
    if data.hematuria:
        summary += " 혈뇨가 동반되어 사구체성 단백뇨 가능성이 있습니다."
    return summary

explain_proteinuria_summary_tool = StructuredTool.from_function(
    func=explain_proteinuria_summary_func,
    name="explain_proteinuria_summary",
    description="나이, 성별, dipstick, UPCR 등 종합정보 기반 단백뇨 해석 요약"
)


protein_tools = [
	screen_proteinuria_dipstick_tool,
	classify_proteinuria_tool,
	check_transient_or_orthostatic_tool,
	suggest_proteinuria_etiology_tool,
	explain_proteinuria_summary_tool
]

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
	[
		("system", "You are a helpful assistant"),
		MessagesPlaceholder("chat_history", optional=True),
		("human", "{input}"),
		MessagesPlaceholder("agent_scratchpad"),
	]
)
from langchain.agents import AgentExecutor, create_openai_functions_agent

# 1. LLM 준비 (gpt-4 or gpt-3.5-turbo)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 2. Agent 생성 (LLM + Tools)
agent = create_openai_functions_agent(
    llm=llm,
    tools=protein_tools,
    prompt=prompt,  # 기본 프롬프트 사용
)

# 3. AgentExecutor 준비
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=protein_tools,
    verbose=True
)
questions = [
    "건강검진에서 소변 단백 2+ 나왔고, 혈뇨도 있다고 합니다. 괜찮은 건가요?"
 
]

# 4. 실행
for question in questions:
    print('\n\n')
    result = agent_executor.invoke({"input": question})
    print()
    print(result['output'])







> Entering new AgentExecutor chain...

Invoking: `screen_proteinuria_dipstick` with `{'data': {'dipstick': '2+'}}`


2+는 의미 있는 단백뇨일 가능성이 있어 정량검사가 필요합니다.
Invoking: `explain_proteinuria_summary` with `{'data': {'age': 0, 'gender': 'unknown', 'dipstick': '2+', 'hematuria': True}}`


0세 여성 환자에서 dipstick 2+ 검사 결과 중등도 이상 단백뇨로 보입니다. 혈뇨가 동반되어 사구체성 단백뇨 가능성이 있습니다.소변 검사에서 단백질 2+와 혈뇨가 동시에 발견된 경우, 이는 중등도 이상의 단백뇨를 의미하며, 사구체성 단백뇨의 가능성이 있습니다. 이는 신장에 문제가 있을 수 있음을 시사하므로 추가적인 정량검사와 전문의의 평가가 필요합니다. 정량검사를 통해 단백뇨의 정확한 수준을 파악하고, 원인을 규명하기 위한 추가 검사가 필요할 수 있습니다. 건강 상태에 대한 정확한 평가를 위해 의사와 상담하시기 바랍니다.

> Finished chain.

소변 검사에서 단백질 2+와 혈뇨가 동시에 발견된 경우, 이는 중등도 이상의 단백뇨를 의미하며, 사구체성 단백뇨의 가능성이 있습니다. 이는 신장에 문제가 있을 수 있음을 시사하므로 추가적인 정량검사와 전문의의 평가가 필요합니다. 정량검사를 통해 단백뇨의 정확한 수준을 파악하고, 원인을 규명하기 위한 추가 검사가 필요할 수 있습니다. 건강 상태에 대한 정확한 평가를 위해 의사와 상담하시기 바랍니다.


In [24]:
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4o")

agent = create_tool_calling_agent(
    llm = llm,
    tools = protein_tools,
    prompt = prompt
)

agent_executor = AgentExecutor(
    agent = agent,
    tools = protein_tools,
    verbose = True,
    handle_parsing_errors= True,
)

questions = [
    """21세 남자가 2주 전부터 갑자기 온 몸이 부어서 왔다. 체중이 7 kg 증가하였다고 한다. 혈압 130/80 mmHg, 맥박 65회/분, 호흡 14회/분, 체온 36.5℃였다. 눈 주위가 부어 있었고 양쪽 다리에 오목부종이 있었다. 검사 결과
는 다음과 같았다. 진단은?
혈액: 혈액요소질소 14 mg/dL, 크레아티닌 0.5 mg/dL, 총단백질 5.3 g/dL, 알부민 2.7 g/dL, 
총콜레스테롤 463 mg/dL, C3 126 mg/dL (참고치, 83 ~ 177), C4 21 mg/dL (참고치, 16 ~ 47) 
소변: 잠혈 (-), 단백질 (4+), 적혈구 0~2/고배율시야, 단백질 10.2 g/일
제시된 function을 활용하시오
""" 
]

# 4. 실행
for question in questions:
    print('\n\n')
    result = agent_executor.invoke({"input": question})
    print()
    print(result['output'])







> Entering new AgentExecutor chain...

Invoking: `screen_proteinuria_dipstick` with `{'data': {'dipstick': '4+'}}`


4+는 의미 있는 단백뇨일 가능성이 있어 정량검사가 필요합니다.
Invoking: `classify_proteinuria` with `{'data': {'upcr': 10200}}`


Nephrotic-range proteinuria입니다.
Invoking: `suggest_proteinuria_etiology` with `{'data': {'hematuria': False, 'upep_result': 'nephrotic'}}`


단백뇨 원인을 특정하기 어렵습니다 가능성이 있습니다. (혈뇨는 없음)
Invoking: `explain_proteinuria_summary` with `{'data': {'age': 21, 'gender': 'male', 'dipstick': '4+', 'upcr': 10200, 'hematuria': False}}`


21세 남성 환자에서 dipstick 4+ 검사 결과 중등도 이상 단백뇨로 보입니다. UPCR 10200.0 mg/gCr로, nephrotic 수준의 단백뇨입니다.제시된 정보와 검사 결과를 바탕으로, 환자는 "신증후군" (Nephrotic Syndrome)이 의심됩니다. 주요한 소견은 다음과 같습니다:

1. Dipstick 검사에서 4+의 단백뇨가 나타났으며, 이는 정량검사가 필요함을 의미합니다.
2. 하루 10.2g의 단백뇨로, 이는 신증후군의 범위에 해당하는 고도의 단백뇨입니다 (nephrotic-range proteinuria).
3. 혈액 검사에서 알부민이 2.7 g/dL로 저알부민혈증이 나타났고, 총콜레스테롤이 463 mg/dL로 고지질혈증이 확인됩니다.
4. 혈뇨는 없으며, 단백뇨의 원인을 특정하기는 어려운 상황입니다.

이 모든 소견은 신증후군의 전형적인 임상 양상에 부합합니다. 따라